<font color='tomato'><font color="#CC3D3D"><p>
# Multi-VAE: Multinomial Variational Autoencoder

- 이 노트북에서는 이해하기 쉬운 구현을 위해 원 논문과 달리 `KL annealing`을 사용하지 않고 `Epoch 당 NDCG`도 측정하지 않았음

<img align='left' src='http://drive.google.com/uc?export=view&id=18eJwXBwp_Dwj9j70Dhbg9_Zys37IbVuk' width=800>

##### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import itertools
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Input, Model, losses, optimizers, initializers

# 4주차에 제공한 msr.zip(ms recommenders 패키지 일부) 사용 시 아래 절차에 따라야 함.
import sys
sys.path.append('/home/work/yhcho/2023-02/RS')  # 본인이 msr.zip 압축을 푼 위치를 확인(셀에서 pwd 명령어 실행) 후 변경해야 함. 
                                                # 윈도우에서는 폴더 구분자를 // 또는 \\로 해야 함.  
from msr.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from msr.constants import (
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
)

### Load & Preprocess Data

프로젝트의 범위에는 사용자와 항목 간의 상호 작용과 1에서 5까지의 정수 등급으로 구성된 MovieLens 데이터 세트가 사용됨. 무비렌즈를 이진화된 클릭 매트릭스(1: 사용자가 이 영화를 좋아함, 0: 사용자가 이 영화를 좋아하지 않거나 시청/평가하지 않음)로 변환하고, 홀드아웃 사용자 데이터를 기반으로 평가함.
  - 평점 3.5 미만의 사용자와 영화 간 상호작용(평점)은 제외
  - 5개 미만의 영화를 클릭한 사용자는 제외
  - 어떤 사용자도 클릭하지 않은 영화도 제외

훈련/검증 세트는 모든 훈련/검증 사용자의 전체 히스토리를 포함하는 클릭 행렬 형태로 모델에 입력되나, 테스트 세트는 다시 훈련과 테스트 부분으로 분할해야 함. 결과적으로 4개의 데이터 세트를 생성:
 - train
 - valid
 - test_tr
 - test_te (with the original ratings)

학습된 모델에 'test_tr'을 넣어 만들어지는 'reconstructed_test_tr' 데이터를 아래의 지표를 사용하여 'test_te'와 비교:
 - MAP@k
 - NDCG@k
 - Recall@k
 - Precision@k

In [ ]:
train, valid, test_tr, test_te = pd.read_pickle('MultiVAE_data.pkl')

### Parameter Setting

In [ ]:
ITEM_DIM = train.shape[1] # Number of items
INTERMEDIATE_DIM = 600    # Dimension of intermediate space
LATENT_DIM = 200          # Dimension of latent space
DROP_RATE = 0.5           # Dropout percentage of the encoder
BETA = 0.1                # A constant parameter β in the ELBO function, when you are not using annealing (annealing=False)
TOP_K = 10                # Number of top k items per user
EPOCHS = 200
BATCH_SIZE = 100

### Build the Multi-VAE

##### Encoder

In [ ]:
encoder_input = Input(shape=(ITEM_DIM,), name="encoder_input")
x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(encoder_input)  # output = x / sqrt(max(sum(x**2), epsilon))
x = layers.Dropout(DROP_RATE)(x)
x = layers.Dense(INTERMEDIATE_DIM, activation="tanh", # 논문에서는 activation 함수로 tanh 사용
                 kernel_initializer=initializers.GlorotUniform(), # Xavier Initialization for weights
                 bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.001) # Normal Initialization for Biases
                )(x)
z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)

# Sampling(Reparameterization)
# VAE의 학습과 최적화 과정에서 수치적 안정성과 효율성을 향상시키기 위해 분산 대신 로그분산(log variance)을 사용 
def sampling(args): 
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=0.1)
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon
encoder_output = layers.Lambda(sampling)([z_mean, z_log_var])

##### Decoder

In [ ]:
x = layers.Dense(INTERMEDIATE_DIM, activation="tanh", # 논문에서는 activation 함수로 tanh 사용
                 kernel_initializer=initializers.GlorotUniform(), # Xavier Initialization for weights
                 bias_initializer=initializers.TruncatedNormal(mean=0.0, stddev=0.001) # Normal Initialization for Biases
                )(encoder_output) 
x = layers.Dropout(DROP_RATE)(x)
decoder_output = layers.Dense(ITEM_DIM, activation='linear')(x) # Loss 계산에서 softmax를 사용하기 때문에 activation 안함

##### Full model

In [ ]:
model = Model(encoder_input, decoder_output)

### Train the Multi-VAE

##### Multi-VAE Loss

- Multi-VAE의 Loss = 
Reconstruction loss(Cross entropy loss) + Beta(0~1 사이의 값) * KL Divergence loss

**1. 크로스 엔트로피 손실:**

$$
L = \frac{1}{N} \sum_{j=1}^{N} H(p_j, q_j) = \frac{1}{N} \sum_{j=1}^{N} -\sum_{i} p_j(i) \log q_j(i)
$$

여기서 $p_j(i)$는 $j$번째 데이터 포인트의 실제 레이블의 확률 값, $q_j(i)$는 $j$번째 데이터 포인트에 대한 모델의 예측 확률 값

In [ ]:
def ReconstLoss(y_true, y_pred):
    log_softmax_var = tf.nn.log_softmax(y_pred) # log(q(i)) 계산
    neg_ll = -tf.reduce_mean(                   # −1/N*∑p(i)*log(q(i)) 계산
        tf.reduce_sum(                          # ∑p(i)*log(q(i)) 계산
            log_softmax_var * y_true,           # p(i)*log(q(i)) 계산
            axis=-1)
    )
    return neg_ll
#    return ITEM_DIM * keras.metrics.binary_crossentropy(y_true, y_pred)  # 출력층 softmax일 경우

# 위 ReconstLoss를 기본 loss로 설정
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss=ReconstLoss)

**2. KL 발산 손실:**
$$ \text{KL loss} = -\frac{1}{2} \times \sum_{i} \left( 1 + z_{\text{log_var}, i} - z_{\text{mean}, i}^2 - \exp(z_{\text{log_var}, i}) \right) $$

- KL발산(KL divergence)은 한 확률분포가 다른 분포와 얼마나 다른지를 측정하는 방법
- VAE에서 평균이 z_mean이고 분산이 z_log_var인 정규분포가 표준정규분포와 얼마나 다른지 측정 
- 표준정규분포에서 크게 벗어난 z_mean과 z_log_var 변수로 인코딩하는 네트워크에 penalty(규제)를 부여
- 재구성 손실(reconstruction loss)에 KL발산을 더해야 함 

In [ ]:
# KL divergence loss
kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)

# Final Loss = Reconstruction loss(model.compile에서 지정) + Beta * KL divergence loss(model.add_loss에서 지정)
model.add_loss(BETA * tf.reduce_mean(kl_loss))

##### Fitting

In [ ]:
%%time
hist = model.fit(train, train, validation_data=(valid, valid), epochs=EPOCHS, batch_size=BATCH_SIZE)

In [ ]:
# Plot Learning Curves
plt.plot(hist.history['loss'], label="train")
plt.plot(hist.history['val_loss'], label="valid")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Evaluate the VAE

In [ ]:
# Copyright (c) Recommenders contributors.
# Licensed under the MIT License.

def recommend_k_items(model, x, k, remove_seen=True):
    """Returns the top-k items ordered by a relevancy score.
    Obtained probabilities are used as recommendation score.

    Args:
        x (numpy.ndarray, int32): rating matrix.
        k (scalar, int32): the number of items to recommend.
    Returns:
        numpy.ndarray, float: A sparse matrix containing the top_k elements ordered by their score.
    """
    # obtain scores
    score = model.predict(x)

    if remove_seen:
        # if true, it removes items from the train set by setting them to zero
        seen_mask = np.not_equal(x, 0)
        score[seen_mask] = 0
    # get the top k items
    top_items = np.argpartition(-score, range(k), axis=1)[:, :k]
    # get a copy of the score matrix
    score_c = score.copy()
    # set to zero the k elements
    score_c[np.arange(score_c.shape[0])[:, None], top_items] = 0
    # set to zeros all elements other then the k
    top_scores = score - score_c
    return top_scores

def map_back_sparse(x, kind):
    """Map back the rating matrix to a pd dataframe

    Args:
        x (numpy.ndarray, int32): rating matrix
        kind (string): specify if the output values are ratings or predictions
    Returns:
        pandas.DataFrame: the generated pandas dataframe
    """
    m, n = x.shape

    # 1) Create a DF from a sparse matrix
    # obtain the non zero items
    items = [np.asanyarray(np.where(x[i, :] != 0)).flatten() for i in range(m)]
    ratings = [x[i, items[i]] for i in range(m)]  # obtain the non-zero ratings

    # Creates user ids following the DF format
    userids = []
    for i in range(0, m):
        userids.extend([i] * len(items[i]))

    # Flatten the lists to follow the DF input format
    items = list(itertools.chain.from_iterable(items))
    ratings = list(itertools.chain.from_iterable(ratings))

    if kind == "ratings":
        col_out = DEFAULT_RATING_COL
    else:
        col_out = DEFAULT_PREDICTION_COL

    # create a df
    out_df = pd.DataFrame.from_dict(
        {DEFAULT_USER_COL: userids, DEFAULT_ITEM_COL: items, col_out: ratings}
    )
    return out_df    

In [ ]:
# Model prediction on the training part of test set 
top_k =  recommend_k_items(model, test_tr, k=TOP_K, remove_seen=True)

# Convert sparse matrix back to df
top_k_df = map_back_sparse(top_k, kind='prediction')
test_df = map_back_sparse(test_te, kind='ratings') # use 'test_te' with the original ratings

# Use the ranking metrics for evaluation
eval_map = map_at_k(test_df, top_k_df, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test_df, top_k_df, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test_df, top_k_df, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test_df, top_k_df, col_prediction='prediction', k=TOP_K)

print("MAP@10:\t\t%f" % eval_map,
      "NDCG@10:\t%f" % eval_ndcg,
      "Precision@10:\t%f" % eval_precision,
      "Recall@10: \t%f" % eval_recall, sep='\n')

<font color='tomato'><font color="#CC3D3D"><p>
# End